In [87]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

In [88]:
import pandas as pd
import requests as rq
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
from pandasgui import show
import numpy as np
from io import BytesIO
from folium.plugins import HeatMap
import nbconvert
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from script import process_data
from script import geolocaliser
from script import modeling
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [89]:
df = pd.read_csv("data/transactions_with_batiment.csv",encoding="utf-8",sep=";")

C:\Users\flori\AppData\Local\Temp\ipykernel_20368\2809893588.py:1: DtypeWarning:

Columns (13,15,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.



In [90]:
df['avoir_un_terrain'] = (df['surface_terrain'] > 0).astype(int)

In [91]:
# Décompte total des transactions par commune
decompte_communes = df['nom_commune'].value_counts()

# Décompte des transactions inondables par commune
decompte_inondables = df[df['zone_inondable'] == 1]['nom_commune'].value_counts()

# Convertir les deux décomptes en DataFrames
decompte_communes_df = decompte_communes.reset_index()
decompte_communes_df.columns = ['nom_commune', 'transactions']

decompte_inondables_df = decompte_inondables.reset_index()
decompte_inondables_df.columns = ['nom_commune', 'transactions_inondables']

# Fusionner les deux DataFrames sur le nom de la commune
decompte_final = pd.merge(decompte_communes_df, decompte_inondables_df, on='nom_commune', how='left')

# Remplacer les valeurs NaN par 0 (communes sans transactions inondables)
decompte_final['transactions_inondables'] = decompte_final['transactions_inondables'].fillna(0).astype(int)

# Fusionner avec le DataFrame initial pour conserver les autres colonnes de `df` (optionnel)
df = pd.merge(df, decompte_final, on='nom_commune', how='left')


In [92]:
# df = df[(df['transactions'] > 5) & (df['transactions_inondables'] > 2)]


In [93]:
# Calcul de la distance pour chaque ligne du DataFrame
df['distance_centre_ville'] = df.apply(
    lambda row: modeling.distance_haversine(
        row['latitude'], row['longitude'],
        row['latitude_mairie'], row['longitude_mairie']
    ),
    axis=1
)

In [94]:
import ast

# Conversion des chaînes en listes de tuples
df["beach_coordinates"] = df["beach_coordinates"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

df["station"] = df["station"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [95]:
def nettoyer_coordinates(coords):
    if isinstance(coords, list):  # Si c'est déjà une liste, la garder
        return coords
    if isinstance(coords, float) or coords is None:  # Si c'est un NaN ou autre, retourner None
        return None
    return coords  # Par défaut, retourner la valeur initiale (si elle est correcte)

df["beach_coordinates"] = df["beach_coordinates"].apply(nettoyer_coordinates)
df["station"] = df["station"].apply(nettoyer_coordinates)

In [96]:
# Calcul de la distance minimale à la plage ou au port en fonction de la disponibilité des données
df["distance_min_beach"] = df.apply(
    lambda row: modeling.distance_minimale(row["latitude"], row["longitude"], row["beach_coordinates"])
    if row["beach_coordinates"] else None,  # Si aucune coordonnée plage disponible, retourner None
    axis=1
)

# Si distance_min_beach est NaN, calculer la distance au port
df["distance_min_beach"] = df.apply(
    lambda row: modeling.distance_haversine(
        row['latitude'], row['longitude'],
        row['latitude_port'], row['longitude_port']
    ) if pd.isna(row["distance_min_beach"]) else row["distance_min_beach"],
    axis=1
)


In [97]:
df["distance_min_station"] = df.apply(
    lambda row: modeling.distance_minimale(row["latitude"], row["longitude"], row["station"])
    if row["station"] else None,  # Si aucune coordonnée disponible, retourner None
    axis=1
)

In [98]:
def convertir_en_numerique(df, colonnes):
    """
    Convertit les colonnes spécifiées du DataFrame en type numérique.
    
    Args:
        df (pd.DataFrame): Le DataFrame à modifier.
        colonnes (list): Liste des colonnes à convertir en numériques.
        
    Returns:
        pd.DataFrame: Le DataFrame avec les colonnes converties.
    """
    for col in colonnes:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

In [99]:
# Affichage des modalités de 'code_scenario'
modalites_code_scenario = df['code_scenario'].value_counts()
print(modalites_code_scenario)

code_scenario
02Moy    3199
04Fai    2707
01For    1324
03Mcc    1070
Name: count, dtype: int64


In [100]:
# Définir les bornes des tranches et les labels
bins = [0, 10000, 50000, np.inf]  # Définir les bornes des tranches
labels = ['0-10000','10000-50000', 'plus_50000']  # Labels des tranches

# Créer la variable catégorielle 'pop'
df['pop_cut'] = pd.cut(df['Population'], bins=bins, labels=labels, right=False)

# Créer des variables dummies pour les catégories de population
df_dummies_population = pd.get_dummies(df['pop_cut'], prefix='population_',drop_first=True)
df_dummies_population = df_dummies_population.astype(int)

# Ajouter les dummies au DataFrame
df = pd.concat([df, df_dummies_population], axis=1)

In [101]:
# Fonction pour gérer la fusion des modalités et la création de dummies
def gerer_code_scenario(df, fusionner_modalites=True):
    # Si fusionner_modalites est True, fusionner les modalités '02Moy' et '03Mcc'
    if fusionner_modalites:
        df['code_scenario'] = df['code_scenario'].replace({
            '02Moy': '02Moy_03Mcc',
            '03Mcc': '02Moy_03Mcc'
        })
    
    # Création des dummies pour la colonne 'code_scenario'
    if 'code_scenario' in df.columns:  # Vérifie que la colonne existe
        df_dummies_scenario = pd.get_dummies(df['code_scenario'], prefix='scenario')
        df_dummies_scenario = df_dummies_scenario.astype(int)
    else:
        df_dummies_scenario = pd.DataFrame()  # Créer un DataFrame vide si aucune colonne
    
    # Ajouter les dummies au DataFrame original
    df = pd.concat([df, df_dummies_scenario], axis=1)
    
    return df

# Utilisation de la fonction
df = gerer_code_scenario(df, fusionner_modalites=True)  # Mettre False pour ne pas fusion


In [102]:
print(df['code_scenario'].value_counts(dropna=False))

code_scenario
NaN            72177
02Moy_03Mcc     4269
04Fai           2707
01For           1324
Name: count, dtype: int64


In [103]:
# Création des dummies pour la colonne 'departmentName'
df_dummies_communes = pd.get_dummies(df['nom_commune'], prefix='commune')
# # Si vous souhaitez supprimer une modalit/é particulière, vous pouvez utiliser .drop() :
df_dummies_communes = df_dummies_communes.drop(columns=['commune_Nice'])

# # Convertir les dummies en entiers (0/1)
df_dummies_communes = df_dummies_communes.astype(int)

# # Ajouter les dummies pour 'departmentName' au DataFrame original
df = pd.concat([df, df_dummies_communes], axis=1)

In [104]:
# Création des dummies pour la colonne 'departmentName'
df_dummies_department = pd.get_dummies(df['departmentName'], prefix='department')
# # Si vous souhaitez supprimer une modalit/é particulière, vous pouvez utiliser .drop() :
df_dummies_department = df_dummies_department.drop(columns=['department_Alpes-Maritimes'])

# # Convertir les dummies en entiers (0/1)
df_dummies_department = df_dummies_department.astype(int)

# # Ajouter les dummies pour 'departmentName' au DataFrame original
df = pd.concat([df, df_dummies_department], axis=1).drop(columns='departmentName')

In [105]:
# Appliquer le remplacement des modalités spécifiées
df['periode_construction_dpe'] = df['periode_construction_dpe'].replace({
    '2013-2021': 'après 2013',
    'après 2021': 'après 2013',
    '2001-2005': '2001-2012',
    '2006-2012': '2001-2012',
    '1983-1988': '1975-1988',
    '1978-1982': '1975-1988',
    '1975-1977': '1975-1988'
})

# Vérifier après remplacement
print(df['periode_construction_dpe'].unique())

# Création des dummies pour la colonne 'periode_construction_dpe'
df_dummies_construction = pd.get_dummies(df['periode_construction_dpe'], prefix='periode_construction_dpe')

# Supprimer la colonne inutile 'periode_construction_dpe_avant 1948' si elle existe
df_dummies_construction = df_dummies_construction.drop(columns=['periode_construction_dpe_avant 1948'], errors='ignore')

# Convertir les dummies en type entier
df_dummies_construction = df_dummies_construction.astype(int)

# Ajouter les dummies au DataFrame original
df = pd.concat([df, df_dummies_construction], axis=1)

[nan '2001-2012' '1989-2000' '1948-1974' '1975-1988' 'avant 1948'
 'après 2013']


In [106]:

df2=df.dropna(subset=['periode_construction_dpe'])

# Vérifier le résultat final
print(df2.head())

       valeur_fonciere  surface_reelle_bati  surface_terrain  nombre_locaux  \
18367         125250.0                 42.0              0.0              2   
18368          89700.0                 21.0              0.0              2   
18369         183550.0                 45.0              0.0              1   
18370          80000.0                 36.0              0.0              2   
18372         212650.0                 84.0              0.0              1   

       nature_culture_sols_ou_vide  nombre_pieces_principales  id_mutation  \
18367                         True                   125250.0  2023-113352   
18368                         True                    89700.0  2023-110189   
18369                         True                   183550.0  2023-110124   
18370                         True                    80000.0  2023-110195   
18372                         True                   212650.0  2023-110078   

      date_mutation  numero_disposition  adresse_numero 

In [107]:
count = df2[df2['nom_commune'] == "Nice"].shape[0]
print(count)

0


In [108]:
# Filtrer les colonnes à conserver
colonnes_a_conserver = [col for col in df2.columns if not col.startswith('commune_')]

# Réassigner le DataFrame avec uniquement les colonnes restantes
df2 = df2[colonnes_a_conserver]

# Création des dummies pour la colonne 'departmentName'
df2_dummies_communes = pd.get_dummies(df2['nom_commune'], prefix='commune')

# # Si vous souhaitez supprimer une modalit/é particulière, vous pouvez utiliser .drop() :
df2_dummies_communes = df2_dummies_communes.drop(columns=['commune_Toulon'])

# # Convertir les dummies en entiers (0/1)
df2_dummies_communes = df2_dummies_communes.astype(int)

# # Ajouter les dummies pour 'departmentName' au DataFrame original
df2 = pd.concat([df2, df2_dummies_communes], axis=1)

In [109]:
show(df.head())

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_20368\3342683114.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_20368\3342683114.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_20368\3342683114.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_20

In [110]:
# Transformation en log de certaines colonnes
variables_continues =    ["surface_reelle_bati",
                          "prix_m2",
                          "distance_min_beach",
                          "distance_min_station",
                          "distance_centre_ville"
                          ]

df = modeling.transformer_log(df, variables_continues)
df2 = modeling.transformer_log(df2, variables_continues)

df_app = df[df["type_local"]=="Appartement"]
df_app2 = df2[df2["type_local"]=="Appartement"]

# Colonnes explicatives, simplifiées à partir de variables_continues

colonnes_geographiques = [
    "log_distance_min_beach",
    "log_distance_min_station",
    "log_distance_centre_ville"
]

colonnes_explicatives1 = [
    "zone_inondable",
    "log_surface_reelle_bati",
    "nombre_pieces_principales"
    # "scenario_01For"
] + [
    col for col in df_app2.columns if col.startswith('population')
] + [
#     col for col in df.columns if col.startswith('scenario')
] + [
#     col for col in df.columns if col.startswith('department')
] + [
    col for col in df.columns if col.startswith('periode_construction_dpe_')
]

colonnes_explicatives2 = colonnes_explicatives1 + [col for col in df_app2.columns if col.startswith('commune_')]

colonnes_explicatives3 = colonnes_explicatives2 + colonnes_geographiques

colonnes_explicatives4 = [
    col for col in colonnes_explicatives3 if col != "zone_inondable"
] + [
    col for col in df_app2.columns if col.startswith("scenario")
    
]

colonne_dependante = "log_prix_m2"

# Création du modèle avec dummies pour les départements
model_app1 = modeling.construire_modele_regression(
    df_app2,
    colonnes_explicatives=colonnes_explicatives1,
    colonne_dependante=colonne_dependante
)

model_app2 = modeling.construire_modele_regression(
    df_app2,
    colonnes_explicatives=colonnes_explicatives2,
    colonne_dependante=colonne_dependante
)

model_app3 = modeling.construire_modele_regression(
    df_app2,
    colonnes_explicatives=colonnes_explicatives3,
    colonne_dependante=colonne_dependante
)

model_app4 = modeling.construire_modele_regression(
    df_app2,
    colonnes_explicatives=colonnes_explicatives4,
    colonne_dependante=colonne_dependante
)

print(model_app3.summary())

                            OLS Regression Results                            
Dep. Variable:            log_prix_m2   R-squared:                       0.566
Model:                            OLS   Adj. R-squared:                  0.561
Method:                 Least Squares   F-statistic:                     111.4
Date:                Wed, 11 Dec 2024   Prob (F-statistic):               0.00
Time:                        04:43:25   Log-Likelihood:                -13950.
No. Observations:               23549   AIC:                         2.845e+04
Df Residuals:                   23276   BIC:                         3.065e+04
Df Model:                         272                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

In [111]:
# Transformation en log de certaines colonnes
variables_continues =    ["surface_reelle_bati",
                          "prix_m2",
                          "distance_min_beach",
                          "distance_min_station",
                          "distance_centre_ville"
                          ]

df = modeling.transformer_log(df, variables_continues)
df2 = modeling.transformer_log(df2, variables_continues)

df_mai = df[df["type_local"]=="Maison"]
df_mai2 = df2[df2["type_local"]=="Maison"]

# Colonnes explicatives, simplifiées à partir de variables_continues

colonnes_geographiques = [
    "log_distance_min_beach",
    "log_distance_min_station",
    "log_distance_centre_ville"
]

colonnes_explicatives1 = [
    "zone_inondable",
    "log_surface_reelle_bati",
    "nombre_pieces_principales",
    'avoir_un_terrain'
    # "scenario_01For"
] + [
    col for col in df_mai2.columns if col.startswith('population')
] + [
#     col for col in df.columns if col.startswith('scenario')
] + [
#     col for col in df.columns if col.startswith('department')
# ] + [
    col for col in df_mai2.columns if col.startswith('periode_construction_dpe_')
]

colonnes_explicatives2 = colonnes_explicatives1 + [col for col in df_mai2.columns if col.startswith('commune_')]

colonnes_explicatives3 = colonnes_explicatives2 + colonnes_geographiques

colonnes_explicatives4 = ["scenario_01For"] + [
    col for col in colonnes_explicatives3 if col != "zone_inondable"
    ] 

colonne_dependante = "log_prix_m2"

# Création du modèle avec dummies pour les départements

model_mai1 = modeling.construire_modele_regression(
    df_mai2,
    colonnes_explicatives=colonnes_explicatives1,
    colonne_dependante=colonne_dependante
)

model_mai2 = modeling.construire_modele_regression(
    df_mai2,
    colonnes_explicatives=colonnes_explicatives2,
    colonne_dependante=colonne_dependante
)

model_mai3 = modeling.construire_modele_regression(
    df_mai2,
    colonnes_explicatives=colonnes_explicatives3,
    colonne_dependante=colonne_dependante
)

model_mai4 = modeling.construire_modele_regression(
    df_mai2,
    colonnes_explicatives=colonnes_explicatives4,
    colonne_dependante=colonne_dependante
)

print(model_mai3.summary())


                            OLS Regression Results                            
Dep. Variable:            log_prix_m2   R-squared:                       0.647
Model:                            OLS   Adj. R-squared:                  0.638
Method:                 Least Squares   F-statistic:                     75.74
Date:                Wed, 11 Dec 2024   Prob (F-statistic):               0.00
Time:                        04:43:28   Log-Likelihood:                -7865.6
No. Observations:               15931   AIC:                         1.649e+04
Df Residuals:                   15554   BIC:                         1.938e+04
Df Model:                         376                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

In [119]:
# Fonction pour arrondir les p-values et ajouter des étoiles de significativité
def arrondir_pvalue_ajouter_etoiles(resultats, model_ids):
    for model_id in model_ids:
        pvalue_col = f"{model_id}_pvalue"
        
        # Convertir les p-values en float (si possible) avant l'arrondi
        resultats[pvalue_col] = pd.to_numeric(resultats[pvalue_col], errors='coerce')
        
        # Appliquer un arrondi à 3 chiffres et ajouter des étoiles de significativité
        resultats[pvalue_col] = resultats[pvalue_col].apply(
            lambda p: f"{round(p, 3)}***" if p < 0.01 else (f"{round(p, 3)}**" if p < 0.05 else (f"{round(p, 3)}*" if p < 0.1 else f"{round(p, 3)}")) if pd.notna(p) else ""
        )
        
    return resultats

# Fonction pour extraire les résultats des modèles
def extraire_resultats_modele(modele, suffixe):
    # Extraction du nombre d'observations et du R² ajusté
    n_obs = modele.nobs  # Nombre d'observations
    r2_adj = modele.rsquared_adj  # R² ajusté
    resultats = pd.DataFrame({
        "variable": modele.params.index,
        f"{suffixe}_coef": modele.params.values,
        f"{suffixe}_pvalue": modele.pvalues.values  # Remplacer 'signif' par 'pvalue'
    })
    
    # Ajouter les informations en haut du DataFrame
    info = pd.DataFrame({
        "variable": ["Observations", "R² ajusté"],
        f"{suffixe}_coef": [int(round(n_obs)), f"{r2_adj:.4f}"],  # Convertir Observations en entier et arrondir le R² ajusté à 4 chiffres
        f"{suffixe}_pvalue": ["", ""]
    })
    
    # Fusionner l'en-tête avec les résultats
    resultats = pd.concat([info, resultats], ignore_index=True)
    return resultats

# Extraction des résultats pour les modèles app
resultats_app1 = extraire_resultats_modele(model_app1, "A.1")
resultats_app2 = extraire_resultats_modele(model_app2, "A.2")
resultats_app3 = extraire_resultats_modele(model_app3, "A.3")
resultats_app4 = extraire_resultats_modele(model_app4, "A.4")

# Extraction des résultats pour les modèles mai
resultats_mai1 = extraire_resultats_modele(model_mai1, "M.1")
resultats_mai2 = extraire_resultats_modele(model_mai2, "M.2")
resultats_mai3 = extraire_resultats_modele(model_mai3, "M.3")
resultats_mai4 = extraire_resultats_modele(model_mai4, "M.4")

# Filtrer les résultats pour exclure les variables qui commencent par 'commune'
def filtrer_variables(resultats):
    return resultats[~resultats["variable"].str.startswith("commune")]

resultats_app1 = filtrer_variables(resultats_app1)
resultats_app2 = filtrer_variables(resultats_app2)
resultats_app3 = filtrer_variables(resultats_app3)
resultats_app4 = filtrer_variables(resultats_app4)

resultats_mai1 = filtrer_variables(resultats_mai1)
resultats_mai2 = filtrer_variables(resultats_mai2)
resultats_mai3 = filtrer_variables(resultats_mai3)
resultats_mai4 = filtrer_variables(resultats_mai4)

# Effectuer les jointures (full join) sur la colonne 'variable'
tableau_app = resultats_app1.merge(resultats_app2, on="variable", how="outer")
tableau_app = tableau_app.merge(resultats_app3, on="variable", how="outer")
tableau_app = tableau_app.merge(resultats_app4, on="variable", how="outer")

tableau_mai = resultats_mai1.merge(resultats_mai2, on="variable", how="outer")
tableau_mai = tableau_mai.merge(resultats_mai3, on="variable", how="outer")
tableau_mai = tableau_mai.merge(resultats_mai4, on="variable", how="outer")

# Convertir les coefficients en float, puis arrondir à 4 chiffres après la virgule
coef_columns_app = [col for col in tableau_app.columns if "_coef" in col]
coef_columns_mai = [col for col in tableau_mai.columns if "_coef" in col]

# Convertir en float et arrondir
tableau_app[coef_columns_app] = tableau_app[coef_columns_app].apply(pd.to_numeric, errors='coerce')
tableau_mai[coef_columns_mai] = tableau_mai[coef_columns_mai].apply(pd.to_numeric, errors='coerce')

# Appliquer l'arrondi et les étoiles aux p-values
model_ids_app = [col.split("_")[0] for col in tableau_app.columns if "_pvalue" in col]
model_ids_mai = [col.split("_")[0] for col in tableau_mai.columns if "_pvalue" in col]

tableau_app = arrondir_pvalue_ajouter_etoiles(tableau_app, model_ids_app)
tableau_mai = arrondir_pvalue_ajouter_etoiles(tableau_mai, model_ids_mai)

# Liste des variables dans l'ordre souhaité
ordre_variables_app = [
    'Observations',
    'R² ajusté',
    'const',
    'zone_inondable',
    'scenario_04Fai',
    'scenario_02Moy_03Mcc',
    'scenario_01For',
    'log_distance_centre_ville',
    'log_distance_min_beach',
    'log_distance_min_station',
    'log_surface_reelle_bati',
    'nombre_pieces_principales',
    'periode_construction_dpe_1948-1974',
    'periode_construction_dpe_1975-1988',
    'periode_construction_dpe_1989-2000',
    'periode_construction_dpe_2001-2012',
    'periode_construction_dpe_après 2013',
    'population__10000-50000',
    'population__plus_50000'
]

ordre_variables_mai = [
    'Observations',
    'R² ajusté',
    'const',
    'zone_inondable',
    'scenario_01For',
    'log_distance_centre_ville',
    'log_distance_min_beach',
    'log_distance_min_station',
    'log_surface_reelle_bati',
    'nombre_pieces_principales',
    'periode_construction_dpe_1948-1974',
    'periode_construction_dpe_1975-1988',
    'periode_construction_dpe_1989-2000',
    'periode_construction_dpe_2001-2012',
    'periode_construction_dpe_après 2013',
    'population__10000-50000',
    'population__plus_50000'
]

# Fonction pour ajouter une ligne vide entre "R² ajusté" et "const"
def ajouter_ligne_vide(tableau):
    # Trouver l'index de "R² ajusté"
    r2_adj_index = tableau[tableau['variable'] == 'R² ajusté'].index[0]
    
    # Créer une ligne vide
    ligne_vide = pd.DataFrame({'variable': [''], 'coef': [''], 'pvalue': ['']}, index=[r2_adj_index + 1])
    
    # Ajouter la ligne vide au DataFrame
    tableau = pd.concat([tableau.iloc[:r2_adj_index + 1], ligne_vide, tableau.iloc[r2_adj_index + 1:]]).reset_index(drop=True)
    
    return tableau

# Fonction pour réorganiser les lignes selon l'ordre des variables et ajouter une ligne vide entre R² ajusté et const
def reordonner_lignes(tableau, ordre_variables):
    # Vérifier que toutes les variables de l'ordre sont présentes dans le DataFrame
    variables_presentes = [var for var in ordre_variables if var in tableau['variable'].values]
    
    # Trier les lignes en fonction de l'ordre des variables
    tableau_reorganise = tableau.set_index("variable").loc[variables_presentes].reset_index()
    
    # Ajouter la ligne vide entre "R² ajusté" et "const"
    tableau_reorganise = ajouter_ligne_vide(tableau_reorganise)
    
    return tableau_reorganise

# Réorganiser les lignes pour les deux tableaux (app et mai)
tableau_app = reordonner_lignes(tableau_app, ordre_variables_app)
tableau_mai = reordonner_lignes(tableau_mai, ordre_variables_mai)

# Renommer les colonnes des coefficients avec M1, M2, M3, M4 en fonction des modèles
def renommer_coef_colonnes(tableau, prefixe):
    # Renommer les colonnes des coefficients
    for i, col in enumerate([col for col in tableau.columns if "_coef" in col]):
        tableau.rename(columns={col: f"({prefixe} - M{i+1})"}, inplace=True)
    
    # Supprimer les titres des colonnes contenant "pvalue"
    tableau = tableau.rename(columns={col: "" for col in tableau.columns if "pvalue" in col})
    
    return tableau

# Appliquer le renommage des colonnes pour chaque tableau
tableau_app = renommer_coef_colonnes(tableau_app, "App")
tableau_mai = renommer_coef_colonnes(tableau_mai, "Maison")

In [120]:
# Appliquer le style pour masquer l'index
tableau_app_display = tableau_app.fillna("").style.hide(axis="index")

# Afficher le tableau sans NaN et sans index
tableau_app_display

variable,(App - M1),,(App - M2),,(App - M3),,(App - M4),,coef,
Observations,25957.000000,,25957.000000,,23549.000000,,23549.000000,,,
R² ajusté,0.497200,,0.560700,,0.560500,,0.560700,,,
,,,,,,,,,,
const,10.325674,0.0***,9.943042,0.0***,9.905428,0.0***,9.904443,0.0***,,
zone_inondable,-0.029343,0.001***,-0.027427,0.003***,-0.029188,0.002***,,,,
scenario_04Fai,,,,,,,0.009399,0.569,,
scenario_02Moy_03Mcc,,,,,,,-0.043535,0.0***,,
scenario_01For,,,,,,,-0.041426,0.028**,,
log_distance_centre_ville,,,,,0.004381,0.518,0.004490,0.508,,
log_distance_min_beach,,,,,-0.033362,0.0***,-0.033662,0.0***,,


In [121]:
# Appliquer le style pour masquer l'index
tableau_mai_display = tableau_mai.fillna("").style.hide(axis="index")

# Afficher le tableau sans NaN et sans index
tableau_mai_display

variable,(Maison - M1),,(Maison - M2),,(Maison - M3),,(Maison - M4),,coef,
Observations,18175.000000,,18175.000000,,15931.000000,,15931.000000,,,
R² ajusté,0.517400,,0.649000,,0.638200,,0.638200,,,
,,,,,,,,,,
const,10.765321,0.0***,10.557126,0.0***,10.525246,0.0***,10.526584,0.0***,,
zone_inondable,0.028580,0.023**,0.016462,0.199,0.009366,0.512,,,,
scenario_01For,,,,,,,0.007469,0.823,,
log_distance_centre_ville,,,,,0.016001,0.001***,0.016242,0.001***,,
log_distance_min_beach,,,,,-0.043542,0.0***,-0.043616,0.0***,,
log_distance_min_station,,,,,0.015120,0.001***,0.015022,0.001***,,
log_surface_reelle_bati,-0.735504,0.0***,-0.690512,0.0***,-0.683577,0.0***,-0.683658,0.0***,,
